## Author: Sean Huver

#### Email: huvers@gmail.com

The goal here is to create a generalized format for quickly generating data from various boson sampling configurations using the strawberryfields library. This data will then later be used to train generative ML models with Tensorflow.

The purpose of the ML model is to generate a probability distribution for a given Boson Sampling configuration of input: <b>n</b> photons , <b>m</b> modes, and <b>(m*(m-1)/2)</b> Beam splitters with various configurations, <b>(θ,ϕ)</b>. An ML model that can successfully generate accurate probability distributions for large n and m (where m > n) is equivalent to calculating the matrix permanent (https://en.wikipedia.org/wiki/Permanent_(mathematics)), a difficult problem thought to be NP and more efficiently solvable through quantum mechanics than known classical means (though there is no known killer application for doing so). 

In [7]:
import os
import strawberryfields as sf
from strawberryfields.ops import *
import itertools as it
import numpy as np
import random
import pickle
from tqdm import tqdm

import multiprocessing
from multiprocessing import Process, TimeoutError

In [3]:
class BosonSampler:
    
    def __init__(self, photon_num, modes, verbose=False):
        self.photon_num     = photon_num  # how many total photons in the experiment
        self.modes          = modes       # how many accessible modes
        self.eng            = sf.Engine(backend="fock", backend_options={"cutoff_dim": self.photon_num+1})
        self.boson_sampling = sf.Program(self.modes)
        self.photons        = 0           # counter for deploying photons
        self.mode_num       = 0           # keeps track of which mode we're interacting with
        self.bs_num         = 0           # counter for deploying beam splitters
        self.bs_max         = modes*(modes-1)/2 # total number of beam splitters (to guarantee unitary conditions)
        self.bs_variables   = []          # keep track of BS(θ,ϕ) properties for input to NN
        self.photon_pos     = list(np.zeros(modes)) #  keep track of how many photons occupy the modes (1 for mode w/ photon
                                                    #   0 for mode w/out photon)
        self.input          = []   # all the inputs for the sampler (photon_pos + bs_variables)
        self.output         = []   # the probability distribution output
        self.probs          = []   # all fock state probabilities
        self.outcome_possibilities = [] # different outcome mode possibilities (assuming no loss)
        self.verbose        = verbose 
    
    def run_sampler(self):
        if self.verbose:
            print('Setting up Boson Sampler...')
        with self.boson_sampling.context as q:
            while self.photons < self.photon_num:
                # all photons are placed in modes in ascending order
                Fock(1)|q[self.mode_num]
                # update input list for NN to let it know there was a photon here
                self.photon_pos[self.photons] = 1
                # go to next mode/photon
                self.mode_num +=1
                self.photons  +=1

            while self.mode_num < self.modes:
                Vac     | q[self.mode_num]
                self.mode_num +=1
            self.mode_num = 0
            if self.verbose:
                print('Simulating {} photons bouncing through {} beam splitters in {} modes!'.format(self.photon_num, self.bs_max, self.modes))
            while self.bs_num < self.bs_max:
                if self.mode_num+1 < self.modes:
                    
                    # get random numbers for BS(θ,ϕ) and append to our input list
                    var_1 = random.uniform(0, 1)
                    var_2 = random.uniform(0, 1)
                    self.bs_variables.append(var_1)
                    self.bs_variables.append(var_2)
                    
                    BSgate(var_1, var_2) | (q[self.mode_num], q[self.mode_num+1])
                    self.bs_num +=1
                    self.mode_num +=2
                else:
                    if self.mode_num % 2 == 0:
                        self.mode_num = 1
                    else:
                        self.mode_num = 0
                    continue

        # get total input for the Neural Net
        self.input = self.photon_pos + self.bs_variables
    
        # run the engine
        self.results = self.eng.run(self.boson_sampling)
        self.probs = self.results.state.all_fock_probs()  # get all outcome probabilities
        
        # num_outcome_possibilities = (n+m-1)! / n!(m-1)! 
        self.outcome_possibilities = [config for config in it.product(list(range(self.photon_num+1)), 
                                                                      repeat=self.modes) if sum(config)==self.photon_num]
        # get output probabilities
        for outcome in self.outcome_possibilities:
            self.output.append(self.probs[outcome])

        
    def display_outcomes(self):
        print('Number of outcome possibilities: ', len(self.outcome_possibilities))
        prob_sum = 0
        # display probabilities per possible outcome mode
        for outcome in self.outcome_possibilities:
            print('Outcome: {} -- Probability: {}'.format(outcome, self.probs[outcome]))
            prob_sum += self.probs[outcome]
        print("Sanity Check -- Total Probability = {}".format(prob_sum))

In [13]:
# run a single Boson Sampling simulation -- large n/m consume a lot of RAM!
b_sampler = BosonSampler(photon_num=3, modes=6, verbose=True)
b_sampler.run_sampler()
b_sampler.display_outcomes()

Setting up Boson Sampler...
Simulating 3 photons bouncing through 15.0 beam splitters in 6 modes!
Number of outcome possibilities:  56
Outcome: (0, 0, 0, 0, 0, 3) -- Probability: 2.6604363038775935e-10
Outcome: (0, 0, 0, 0, 1, 2) -- Probability: 6.481734033953618e-08
Outcome: (0, 0, 0, 0, 2, 1) -- Probability: 6.082791734459418e-06
Outcome: (0, 0, 0, 0, 3, 0) -- Probability: 0.0003869826455849814
Outcome: (0, 0, 0, 1, 0, 2) -- Probability: 5.654432769564953e-07
Outcome: (0, 0, 0, 1, 1, 1) -- Probability: 8.402945357574007e-05
Outcome: (0, 0, 0, 1, 2, 0) -- Probability: 0.007740162185304018
Outcome: (0, 0, 0, 2, 0, 1) -- Probability: 0.00028233391182144084
Outcome: (0, 0, 0, 2, 1, 0) -- Probability: 0.048701857754758576
Outcome: (0, 0, 0, 3, 0, 0) -- Probability: 0.09322413845360283
Outcome: (0, 0, 1, 0, 0, 2) -- Probability: 3.095338156200602e-07
Outcome: (0, 0, 1, 0, 1, 1) -- Probability: 2.6218367523240284e-05
Outcome: (0, 0, 1, 0, 2, 0) -- Probability: 0.0016425185279316493
Outcome:

In [8]:
# now generate input/output pairs to save for our ML model
# all data generated is pickle'd


sample_size = 5  # how many input/output data set pairs to generate

photon_num = 3
modes      = 6


X = []
Y = []

for i in tqdm(range(sample_size)):
    #print('Simulating Boson Sample {} out of {}'.format(i, sample_size))
    b_sampler = BosonSampler(photon_num, modes)
    b_sampler.run_sampler()

    X.append(np.array(b_sampler.input))
    Y.append(np.array(b_sampler.output))

filename = '../data/' + str(modes) + '_' + str(photon_num) + '_' + 'data.sav'

if os.path.exists(filename):
    append_write = 'a' # append if already exists
else:
    append_write = 'wb' # make a new file if not
    
pickle.dump([X,Y], open(filename, append_write))

100%|██████████| 5/5 [01:23<00:00, 16.77s/it]


In [14]:
cores = multiprocessing.cpu_count()
filename = '../data/data_new.sav'

sample_size = 200  # how many input/output data set pairs to generate


def run_sim(photon_num, modes):
    b_sampler = BosonSampler(photon_num, modes)
    b_sampler.run_sampler()
    
    X = np.array(b_sampler.input)
    Y = np.array(b_sampler.output)
    
    if os.path.exists(filename):
        append_write = 'a' # append if already exists
    else:
        append_write = 'wb' # make a new file if not
        
    pickle.dump([X,Y], open(filename, append_write))

for j in tqdm(range(sample_size)):
    for i in range(4):
        proc = Process(target=run_sim, args=(3,5))
        proc.start()

100%|██████████| 200/200 [00:18<00:00, 10.75it/s] 
